<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/ath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas pandas-ta numpy matplotlib yfinance
!pip install --upgrade pandas-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 10.7 MB/s eta 0:00:00
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=2809ea5283f18fbb258ea78bf7e6c27adf1471f64d77b1c7583930b0a06e728e
  Stored in directory: /root/.cache/pip/wheels/54/4a/75/06b8e63fce6f6d2c1baae5c208edb18eca128407b0c96e1153
Successfully built pandas-ta
 

In [2]:
import pandas_ta as ta
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import yfinance as yf
from dateutil.relativedelta import relativedelta
from datetime import date as date


In [3]:
# enddate = date.today()
# startdate = enddate - relativedelta(years=4)

# df = yf.download('HINDZINC.NS', start=startdate, end=enddate)


In [4]:
ticker_yahoo = yf.Ticker('HINDZINC.NS')
# last_quote = ticker_yahoo.fast_info['last_price']
# MA200 = ticker_yahoo.fast_info['two_hundred_day_average']
# ticker_yahoo.fast_info

file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)
#print(tickers)

In [5]:
ticker_yahoo.fast_info

lazy-loading dict with keys = ['currency', 'exchange', 'timezone', 'shares', 'market_cap', 'last_price', 'previous_close', 'open', 'day_high', 'day_low', 'regular_market_previous_close', 'last_volume', 'fifty_day_average', 'two_hundred_day_average', 'ten_day_average_volume', 'three_month_average_volume', 'year_high', 'year_low', 'year_change']

In [6]:
from concurrent.futures import ThreadPoolExecutor

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')


def get_stats(ticker):
    info = yf.Tickers(ticker).tickers[ticker].fast_info
    #print(f"{ticker} {info['last_price']} {info['year_high']}")
    return ticker, info['last_price'], info['year_high'], info['two_hundred_day_average']
def paralel_func(ticker):
    with ThreadPoolExecutor(max_workers=None) as executor:
        results = list(executor.map(get_stats, ath_list))
    return results

results = paralel_func(ath_list)


In [7]:
ticker_list = pd.DataFrame(results, columns=['Ticker', 'Last_price', 'ATH', '200EMA']) #creating the dataframe

pd.set_option('display.max_rows', None)
ticker_list = ticker_list.loc[(ticker_list['Last_price'] < ticker_list['ATH'] * 0.95) & (ticker_list['Last_price'] > ticker_list['ATH'] * 0.9) & (ticker_list['Last_price'] > ticker_list['200EMA'])]
ticker_list.reset_index(drop=True, inplace=True)

ticker_list


,Ticker,Last_price,ATH,200EMA
0,ABBOTINDIA.NS,20872.250000,22499.000000,19222.295283
1,ACE.NS,329.549988,354.700012,266.831499
2,AEGISCHEM.NS,367.200012,388.799988,276.333500
3,AKZOINDIA.NS,2210.300049,2400.000000,2041.203253
4,AMARAJABAT.NS,605.049988,668.150024,537.312750
5,APARINDS.NS,1931.099976,2047.949951,1241.254999
6,APTECHT.NS,362.500000,388.700012,283.411500
7,ASHOKLEY.NS,153.899994,169.449997,144.855750
8,ASTRAZEN.NS,3304.050049,3620.000000,3095.267002
9,AXISBANK.NS,882.599976,970.000000,786.500249
